In [22]:
# Importing libraries and packages

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.utils import shuffle
import os
import math
pd.options.plotting.backend = 'plotly'

print(os.getcwd())
from permutation_test_util import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\Users\phuro\UCSD\Project_WHEL\data


In [3]:
# Importing dataset

os.chdir("C:/Users/phuro/UCSD/Project_WHEL/data")
path = os.path.join('interim', 'return_status_by_site.csv')
df = pd.read_csv(path, index_col=0)

In [4]:
df

,site,status
0,AZ,unreturned
1,TX,returned
2,CA,returned
3,CA,returned
4,CA,returned
...,...,...
2856,TX,unreturned
2857,CA,returned
2858,CA,returned
2859,OR,returned


In [33]:
table = df.pivot_table(index='site', columns='status', aggfunc='size')
print(table)
table = table / table.sum()
table

table_log = df.pivot_table(index='site', columns='status', aggfunc='size')
table_log = np.log(table_log)
table_log

status  returned  unreturned
site                        
AZ           296         144
CA          1287         554
OR           194          38
TX           212         136


status,returned,unreturned
site,,
AZ,5.690359,4.969813
CA,7.160069,6.317165
OR,5.267858,3.637586
TX,5.356586,4.912655


In [34]:
# Plot observed distribution

fig = table.plot(kind='barh', title='Distribution of study site by return status (proportions)',
                 barmode='group')
fig.show()

fig = table_log.plot(kind='barh', title='Distribution of study site by return status (log scale)',
                 barmode='group')
fig.show()

In [35]:
result, test_stats = permutation_simulation(df, 10000, shuffle_column='status', 
                                            cats_column='site', 
                                            significance_level=0.01)

0.01777782390302631

STEPS:
shuffled  returned  unreturned
site                          
AZ             310         130
CA            1276         565
OR             156          76
TX             247         101

shuffled  returned  unreturned
site                          
AZ        0.155857    0.149083
CA        0.641528    0.647936
OR        0.078431    0.087156
TX        0.124183    0.115826

shuffled  returned  unreturned
site                          
AZ             NaN   -0.006775
CA             NaN    0.006407
OR             NaN    0.008725
TX             NaN   -0.008357

site
AZ   -0.006775
CA    0.006407
OR    0.008725
TX   -0.008357
Name: unreturned, dtype: float64

site
AZ    0.006775
CA    0.006407
OR    0.008725
TX    0.008357
Name: unreturned, dtype: float64

0.015131964335957829
END

0.0068986074787477925
0.01844548687506052
0.021076932301972773
0.02832839793174388
0.026760139482751445
0.01844548687506055
0.023461031083804998
0.016319112919220877
0.019603807178011197


In [38]:
# sanity check

# Calculate observed test statistic (TVD)
observed = tvd_of_groups(df, groups='status', cats='site')
observed

0.06569619143823138

In [39]:
result

'Obsersed TVD = 0.066, P-value = 0.0012, Reject null hypothesis at 0.01'

In [40]:
test_stats

[0.01777782390302631,
 0.015131964335957829,
 0.01679593267558732,
 0.022726486501446035,
 0.021143813912297485,
 0.016972361751098952,
 0.04269756597063667,
 0.026869686947938447,
 0.011260902855614159,
 0.026092476510717194,
 0.027932873925858276,
 0.04153924566768604,
 0.01447871550407976,
 0.018621915950572228,
 0.027756444850346645,
 0.02114381391229745,
 0.01638599452954554,
 0.019427378102499483,
 0.022917329717113896,
 0.021158228052453647,
 0.011656426861499702,
 0.011723308471824394,
 0.0070750365542594645,
 0.018035548729018795,
 0.010197715877694316,
 0.02272648650144604,
 0.045996674369583185,
 0.03162750633069035,
 0.011370450320801147,
 0.00961134865614089,
 0.03657616892911012,
 0.01960380717801121,
 0.022793368111770726,
 0.00689860747874782,
 0.014560011254560669,
 0.018512368485385247,
 0.029501132374850705,
 0.022726486501446055,
 0.02437604070091927,
 0.027742030710190484,
 0.015213260086438772,
 0.03939845757169015,
 0.03494102886979306,
 0.014478715504079781,
 0.

In [41]:
# Plot observed tvd and simulated tvds

fig = px.histogram(pd.DataFrame({'simulated tvds': test_stats}), 
                x='simulated tvds', nbins=55, histnorm='probability', 
                title=f'Empirical Distribution of TVD <br><sup>{result} significance level</sup>')
fig.add_vline(x=observed, line_color='rgb(0,100,80)', 
              annotation_text='observed', annotation_position='top right')
p_99 = np.percentile(test_stats, 99)
fig.add_vline(x=p_99, line_color='rgb(0,176,246)', 
              annotation_text='significance level', annotation_position='top left')
fig.update_layout(xaxis_range=[0, 0.09])

In [12]:
fig.write_html('C:/Users/phuro/UCSD/Project_WHEL/hypothesis_tests/results.html')